In [1]:
#import sklearnx to speed up code (must be imported before sklearn)

from sklearnex import patch_sklearn
patch_sklearn() 

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
from os.path import isfile
import numpy as np
from data_handling.image_processing import PrepareData
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
image_size = 32
credentials_location = ".gitignore/credentials_for_marketplace.yml"
images_folder = "data/cleaned_images_" + str(image_size)
X_array_folder = "data/X_for_image_"+ str(image_size) + ".npy"
y_array_folder = "data/y_for_image_"+ str(image_size)+".npy"


In [4]:
# Load in the X and y arrays

if isfile(X_array_folder) and isfile(y_array_folder): 
    X = np.load(X_array_folder)
    y = np.load(y_array_folder)
else:
    pipeline = PrepareData()
    product_df, image_df = pipeline.retrieve_dataframes(credentials_location)
    X, y = pipeline.form_arrays(images_folder, image_size)
    pipeline.save_to_files(X_array_folder, y_array_folder)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=False)

In [8]:

model = LogisticRegression(max_iter=100)
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(score)

/home/amy/miniconda3/envs/rec_ranking/lib/python3.10/site-packages/daal4py/sklearn/linear_model/logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.014676715589051963
